## 1. Install dependencies

In [ ]:
pip install transformers torch pandas openpyxl

## 2. Load the Model and Tokenizer

In [1]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Check if GPU is available and set device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Load model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Move model to GPU if available
model.to(device)

# Define labels for interpretation
labels = ["negative", "neutral", "positive"]


## 3. Load the Excel File

In [ ]:
# Load the Excel file
file_path = "your_file.xlsx"  # Change to your file path
df = pd.read_excel(file_path)

# Assuming X_Posts are in a column named 'X_Post'
X_Post = df["X_Post"].astype(str).tolist()


## 4. Process the Tweets and Get Sentiment


In [ ]:
def get_sentiment_batch(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
    return [labels[p] for p in predictions]

# Process in batches (example: batch size 32)
batch_size = 32
df["sentiment"] = [
    sentiment for i in range(0, len(df), batch_size)
    for sentiment in get_sentiment_batch(df["X_Post"][i:i+batch_size].tolist())
]

# Apply sentiment analysis to all X_Posts
df["sentiment"] = df["X_Post"].apply(get_sentiment)


## 5. Save the Results


In [ ]:
# Save the DataFrame with sentiment labels back to an Excel file
output_path = "X_Posts_with_sentiment.xlsx"
df.to_excel(output_path, index=False)

print("Sentiment analysis completed and saved to:", output_path)